In [1]:
!pip install z3-solver
# miniconda base env


   ---------------------------------------- 0.0/58.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/58.4 MB ? eta -:--:--
   ---------------------------------------- 0.1/58.4 MB 1.2 MB/s eta 0:00:49
   ---------------------------------------- 0.5/58.4 MB 3.3 MB/s eta 0:00:18
    --------------------------------------- 0.9/58.4 MB 4.9 MB/s eta 0:00:12
    --------------------------------------- 1.0/58.4 MB 5.1 MB/s eta 0:00:12
   - -------------------------------------- 1.5/58.4 MB 5.2 MB/s eta 0:00:11
   - -------------------------------------- 2.1/58.4 MB 6.3 MB/s eta 0:00:09
   - -------------------------------------- 2.2/58.4 MB 5.7 MB/s eta 0:00:10
   - -------------------------------------- 2.9/58.4 MB 6.8 MB/s eta 0:00:09
   -- ------------------------------------- 3.1/58.4 MB 7.2 MB/s eta 0:00:08
   -- ------------------------------------- 3.4/58.4 MB 6.6 MB/s eta 0:00:09
   -- ------------------------------------- 4.2/58.4 MB 7.6 MB/s eta 0:00:08
   -- -------

In [28]:
from z3 import *

# initialize ints X and Y
# X represents one skin, Y represents another skin
# X1...X5 represent the number of skins used of the skin X for each quality (FN to BS)
# if X3 is 5, that means 5 skins of X in field-tested condition are used in the tradeup
# same goes for Y skin

# A1...A5 are the outputs of skin X
# B1...B5 are the outputs of skin Y

# CX1 represents the cost of skin X1. Same goes for every other skin

# Define variables
X1, X2, X3, X4, X5 = Ints('X1 X2 X3 X4 X5')
Y1, Y2, Y3, Y4, Y5 = Ints('Y1 Y2 Y3 Y4 Y5')

all_skins = [X1,X2,X3,X4,X5,Y1,Y2,Y3,Y4,Y5]

# Example costs for each variable
CX1, CX2, CX3, CX4, CX5 = 1.0, 2.0, 3.0, 4.0, 5.0
CY1, CY2, CY3, CY4, CY5 = 1.5, 2.5, 3.5, 5.5, 4.5

# Setup optimization
opt = Optimize()

# RESTRICTIONS
# the sum of all input skin number variables is 10, because a tradeup involves 10 input skins
cond1 = Sum(all_skins) == 10

# the number of skins used for each skin is higher or equal than 0 and less or equal than 10
cond2 = [ And(skin >= 0, skin <= 10) for skin in all_skins]

opt.add(cond1)
opt.add(cond2)

# Objective function
objective = (CX1*X1 + CX2*X2 + CX3*X3 + CX4*X4 + CX5*X5 + 
             CY1*Y1 + CY2*Y2 + CY3*Y3 + CY4*Y4 + CY5*Y5)

# Maximize the objective function
opt.maximize(objective)

# Check for solution and print it
if opt.check() == sat:
    model = opt.model()
    print("Solution found:")
    for var in all_skins:
        print(f"{var} = {model[var]}")
else:
    print("No solution found.")

Solution found:
X1 = 0
X2 = 0
X3 = 0
X4 = 0
X5 = 0
Y1 = 0
Y2 = 0
Y3 = 0
Y4 = 10
Y5 = 0


In [29]:
!pip install pulp


   ---------------------------------------- 0.0/17.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/17.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/17.7 MB ? eta -:--:--
   ---------------------------------------- 0.2/17.7 MB 1.5 MB/s eta 0:00:12
    --------------------------------------- 0.3/17.7 MB 2.0 MB/s eta 0:00:09
    --------------------------------------- 0.4/17.7 MB 2.0 MB/s eta 0:00:09
   - -------------------------------------- 0.6/17.7 MB 2.4 MB/s eta 0:00:08
   - -------------------------------------- 0.6/17.7 MB 2.4 MB/s eta 0:00:08
   - -------------------------------------- 0.7/17.7 MB 2.2 MB/s eta 0:00:08
   -- ------------------------------------- 0.9/17.7 MB 2.4 MB/s eta 0:00:07
   -- ------------------------------------- 1.1/17.7 MB 2.5 MB/s eta 0:00:07
   -- ------------------------------------- 1.2/17.7 MB 2.6 MB/s eta 0:00:07
   --- ------------------------------------ 1.5/17.7 MB 2.9 MB/s eta 0:00:06
   --- -------------

In [90]:
# trying to solve with pulp to check if it's quicker to solve this type of problems
# using z3 took 36 seconds to optimize the variables multiplied by their respective costs, which is toooo much
# using this solver it took 0.1 seconds which is way better

import pulp as pl

# Create a problem instance
prob = pl.LpProblem("Maximize_Skin_Value", pl.LpMaximize)

# Define variables. Note: PuLP does integer variables by default, we can use LpContinuous for continuous.
# definition of variables and costs for skin X
input_skin_X = pl.LpVariable.dicts("X", range(1,6), lowBound=0, upBound=10, cat='Integer')
input_skin_X_costs = [5.0, 4.0, 3.0, 2.0, 1.0]
input_skin_X_floats = [0, 0.07, 0.15, 0.37, 0.45]

input_skin_Y = pl.LpVariable.dicts("Y", range(1,6), lowBound=0, upBound=10, cat='Integer')
input_skin_Y_costs = [10.0, 8.0, 6.0, 4.0, 2.0]
input_skin_Y_floats = [0, 0.07, 0.15, 0.37, 0.45]

# all_input_skins = input_skin_X.update(input_skin_Y) doesnt work

# these are theoretical minimum floats. They consider the best case scenario when it comes to finding floats
# X1 and Y1 are factory new versions of skins X and Y, respectively. The same goes for the remaining skins

# Objective function
input_cost = pl.lpSum([var_value * input_skin_X_costs[i] for i, var_value in enumerate(input_skin_X.values())]) + \
             pl.lpSum([var_value * input_skin_Y_costs[i] for i, var_value in enumerate(input_skin_Y.values())])
prob += input_cost, "TotalCost"

# Constraint 1: Total skins equals 10
constraint1 = pl.lpSum(input_skin_X.values()) + pl.lpSum(input_skin_Y.values()) == 10
prob += constraint1, "TotalSkins10"

# Constraint 2: The average input floats is higher than 0.2
avg_float = (pl.lpSum([float * skin_amount for float, skin_amount in zip(input_skin_X_floats, input_skin_X.values())]) + \
            pl.lpSum([float * skin_amount for float, skin_amount in zip(input_skin_Y_floats, input_skin_Y.values())])) / 10 # / 10 because the tradeup has 10 input skins
constraint2 = avg_float >= 0.3
prob += constraint2, "MinAvgFloat"

# output constraints
# These are ifs. For example:
# If average input float >= 0.45, then output skin is battle scared (O5)
output_floats_vars = ['O1', 'O2', 'O3', 'O4', 'O5']
lower_floats = [0, 0.07, 0.15, 0.37, 0.45]
upper_floats = [0.07, 0.15, 0.37, 0.45, 1]

# now we need to calculate the probabilities for each output variable
# We need to have a different set of variables for each output skin.
# So each output skin will have a O1,O2,O3,O4,O5 (depending on the available conditions for that output skin)
# For example if 'Redline' has all 5 conditions, there will be output variables O1,O2,O3,O4,O5 for this output skin

output_skins_names_A = ['Redline']
# here we create a list of dictionaries
# each dictionary contains the float variables for a skin
# if there are two possible output skins, there will be two dictionaries in the list, each containing the float variables for the respective output skin
output_skins_floatvars_A = [pl.LpVariable.dicts(skin_name, output_floats_vars, lowBound=0, upBound=1, cat='Integer') for skin_name in output_skins_names_A]
# array with the number of ballots for each of the output skins of X. The ballots of an output skin determine the odds of the trade up resulting in that skin
output_skins_A_Ballots = pl.LpVariable("A_ballots", lowBound=0, upBound=10, cat='Integer')
prob += output_skins_A_Ballots == pl.lpSum(input_skin_X.values())

# array with the probabilities for each of the output skins f X
output_skins_probvars_A = [pl.LpVariable(skin_name+"_prob", lowBound=0, upBound=1, cat='Continuous') for skin_name in output_skins_names_A]

"""
Here we iterate through the list of dictionaries
For each dictionary, we add the restrictions to the solver
The restrictions make it so that only one of the float variables has a value of 1, and the remaining have values of 0
This is because there can't be two output variables of the same skin with different conditions.
We also set upper and lower float boundaries for each variable
"""
for skin_floatvars in output_skins_floatvars_A:
    for index, float_var in enumerate(skin_floatvars.keys()):
        #print(index, float_var)
        prob += skin_floatvars[float_var] <= (avg_float - lower_floats[index]) / 1 + 1 # lower bound restriction
        prob += skin_floatvars[float_var] <= (upper_floats[index] - avg_float) / 1 + 1 # upper bound restriction
    prob += pl.lpSum(skin_floatvars.values()) == 1 # only one output float is allowed (there can't be two outputs of the same skin with different conditions)



# this section is the same code as above. Need to merge it
output_skins_names_B = ['Asimov']
output_skins_floatvars_B = [pl.LpVariable.dicts(skin_name, output_floats_vars, lowBound=0, upBound=1, cat='Integer') for skin_name in output_skins_names_B]
output_skins_B_Ballots = pl.LpVariable("B_ballots", lowBound=0, upBound=10, cat='Integer')
prob += output_skins_B_Ballots == pl.lpSum(input_skin_Y.values()) # the input of this output is Y

# array with the probabilities for each of the output skins f X
output_skins_probvars_B = [pl.LpVariable(skin_name+"_prob", lowBound=0, upBound=1, cat='Continuous') for skin_name in output_skins_names_B]

for skin_floatvars in output_skins_floatvars_B:
    for index, float_var in enumerate(skin_floatvars.keys()):
        #print(index, float_var)
        prob += skin_floatvars[float_var] <= (avg_float - lower_floats[index]) / 1 + 1 # lower bound restriction
        prob += skin_floatvars[float_var] <= (upper_floats[index] - avg_float) / 1 + 1 # upper bound restriction
    prob += pl.lpSum(skin_floatvars.values()) == 1 # only one output float is allowed (there can't be two outputs of the same skin with different conditions)


In [91]:
# Ballots variable definitions and constraints
total_ballots = pl.LpVariable("Total_ballots", lowBound=0, upBound=None, cat='Integer')
ballots_A = pl.lpSum(output_skins_A_Ballots)
ballots_B = pl.lpSum(output_skins_B_Ballots)
constraint3 = total_ballots == ballots_A*len(output_skins_names_A) + ballots_B*len(output_skins_names_B)
prob += constraint3

# constraints for the probabilities of the skins for each output collection
# constraints for the output skins in collection A
for probvar in output_skins_probvars_A:
    prob += probvar == ballots_A / total_ballots

# constraints for the output skins in collection B
for probvar in output_skins_probvars_B:
    prob += probvar == ballots_B / total_ballots


TypeError: object of type 'LpVariable' has no len()

In [92]:
# Assuming a simplified scenario
MAX_BALLOTS = 80  # Define according to your problem's needs
INTERVAL_SIZE = 4  # Size of each interval for approximation
num_intervals = MAX_BALLOTS // INTERVAL_SIZE

# Binary variables for each interval
interval_vars = [pl.LpVariable(f"interval_{i}", cat='Binary') for i in range(num_intervals)]

# Approximate inverse for each interval
approx_inverse = [(1 / ((i * INTERVAL_SIZE + (i+1) * INTERVAL_SIZE) / 2)) for i in range(num_intervals)]

# Ensure only one interval is active
prob += pl.lpSum(interval_vars) == 1

# Constraints and calculation for probvar (for each output skin, adapt as necessary)
for i, probvar in enumerate(output_skins_probvars_A):
    # Calculate probvar as a linear combination, using the active interval's approximated inverse
    prob += probvar == pl.lpSum([ballots_A * approx_inverse[i] * interval_vars[i] for i in range(num_intervals)])
    # Add more constraints as needed to link interval_vars with total_ballots appropriately


TypeError: Non-constant expressions cannot be multiplied

In [ ]:

# Solve the problem
prob.solve()

# Print the status
print("Status:", pl.LpStatus[prob.status])

# get average float of input skins
floats_total = [float * skin_amount.varValue for float, skin_amount in zip(input_skin_X_floats, input_skin_X.values())] + \
               [float * skin_amount.varValue for float, skin_amount in zip(input_skin_Y_floats, input_skin_Y.values())]
print("avg float - ", sum(floats_total) / len(floats_total))

# Print the optimized values of the variables
for v in prob.variables():
    print(v.name, "=", v.varValue)